In [1]:
import pandas as pd
import re
import numpy as np
from timeit import default_timer as timer
from datetime import timedelta
from alive_progress import alive_bar
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#veri ön işleme
def preprocessing(series, remove_hashtag=False, remove_mentions=False, remove_links=False, remove_numbers=False,
                  remove_short_text=False, remove_stopwords=False, lowercase=False, remove_punctuation=False,
                  remove_rare_words=False, rare_limit=5):
    if lowercase:
        print("Lowercasing...")
        start = timer()
        series = series.str.lower()
        print(f"Lowercasing is done in {timedelta(seconds=timer() - start)}")

    if remove_hashtag:
        print("Removing hashtags...")
        start = timer()
        series = series.str.replace(r'((#)[^\s]*)\b', '', regex=True)
        print(f"Removed in {timedelta(seconds=timer() - start)}")

    if remove_mentions:
        print("Removing mentions...")
        start = timer()
        series = series.str.replace(r'((@)[^\s]*)\b', '', regex=True)
        print(f"Removed in {timedelta(seconds=timer() - start)}")

    if remove_links:
        print("Removing links...")
        start = timer()
        series = series.str.replace(r'\n', '', regex=True)
        series = series.apply(lambda x: re.split('https:\/\/.*', str(x))[0])
        print(f"Removed in {timedelta(seconds=timer() - start)}")

    if remove_numbers:
        print("Removing numbers...")
        start = timer()
        series = series.str.replace(r'\d+', '', regex=True)
        print(f"Removed in {timedelta(seconds=timer() - start)}")

    if remove_short_text:
        print("Removing short texts...")
        start = timer()
        series = series.apply(lambda x: re.sub(r'\b\w{1,2}\b', '', x))
        print(f"Removed in {timedelta(seconds=timer() - start)}")

    if remove_stopwords:
        print("Removing stopwords...")
        start = timer()
        with open('assets/stopwords.txt', 'r', encoding='utf-8') as f:
            stop = [line.strip() for line in f]
        series = series.apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop]))
        print(f"Stopwords are removed in {timedelta(seconds=timer() - start)}")

    if remove_punctuation:
        print("Removing punctuation...")
        start = timer()
        series = series.str.replace(r"((')[^\s]*)\b", '', regex=True)
        series = series.str.replace(r'[^\w\s]', '', regex=True)
        print(f"Removed in {timedelta(seconds=timer() - start)}")

    if remove_rare_words:
        print("Removing rare words...")
        start = timer()
        whole_count = pd.Series(" ".join(series).split()).value_counts()
        print(f"There are {whole_count.count()} words in the series")
        print(f"%{round(whole_count[whole_count <= rare_limit].count() / whole_count.count() * 100, 2)} of "
              f"words appear lesser than rare limit ({rare_limit}) times")
        to_remove = whole_count[whole_count <= rare_limit]
        print(f"Removing rare words...")
        series = series.apply(lambda x: " ".join(x for x in x.split() if x not in to_remove))
        print(f"{len(to_remove)} rare words removed")
        print(f"Removed in {timedelta(seconds=timer() - start)}")
    return series

In [3]:
# Duygu analizi fonksiyonu tanımlanıyor. Parametre olarak bir seri ve bir model adı alıyor. Model adı varsayılan olarak "savasy/bert-base-turkish-sentiment-cased" olarak belirleniyor.
def sentiment(series, model_name="savasy/bert-base-turkish-sentiment-cased"):
    # transformers kütüphanesinden gerekli modüller import ediliyor.
    from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
    # label ve score adında iki boş liste oluşturuluyor. Bunlar sonuçları saklamak için kullanılacak.
    label, score = [], []
    # Model adına göre bir sınıflandırma modeli yükleniyor.
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    # Model adına göre bir tokenizer yükleniyor. Tokenizer, metni modelin anlayabileceği şekilde parçalara ayırır.
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # pipeline fonksiyonu ile bir duygu analizi aracı oluşturuluyor. Bu araç, tokenizer ve modeli kullanarak metinleri duygu sınıflarına atar.
    sa = pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)
    # Serinin içindeki her bir metin için bir döngü başlatılıyor. i, metnin indeksi, k ise metnin kendisi oluyor.
    for i, k in enumerate(series):
        # sa aracı ile metnin duygu analizi sonucu alınıyor. Sonuç, bir liste içinde bir sözlük olarak dönüyor. Sözlükte 'label' ve 'score' adında iki anahtar var. 'label', metnin duygu sınıfını, 'score' ise modelin bu sınıfa ne kadar güvendiğini gösteriyor.
        result = sa(k)
        # Sonuçtaki 'label' değerini label listesine ekliyoruz.
        label.append(result[0]['label'])
        # Sonuçtaki 'score' değerini score listesine ekliyoruz.
        score.append(result[0]['score'])
        # Eğer i sayısı 100'ün katı ise, yani 100, 200, 300 gibi ise, ekrana kaç metnin duygu analizinin yapıldığını yazdırıyoruz. Bu, işlemin ilerleyişini takip etmek için kullanılıyor.
        if i % 100 == 0 and i != 0:
            print(f"Sentiment analysis of {i} tweets is done")
    # label ve score listelerini bir veri çerçevesine dönüştürüyoruz ve sütun isimlerini 'label' ve 'score' olarak atıyoruz.
    return pd.DataFrame({'label': label, 'score': score})

In [4]:
def create_tfidf(series):
"""
    Görevler 
    -----
        Verilen serinin tfidf matrisini oluşturur.

    Parametreler
    ----------
        series: pandas.Series
        Dönüştürülecek seri.

    Döndürür
    -------
        df_tfidf_vect: pandas.DataFrame
            Tfidf matrisi.
"""

# TfidfVectorizer sınıfından bir nesne oluşturur. analyzer parametresi, kelime seviyesinde vektörleştirme yapacağını belirtir.
tfidf_vectorized = TfidfVectorizer(analyzer='word')
# Seriyi tfidf ağırlıklı bir matrise dönüştürür.
tfidf_wm = tfidf_vectorized.fit_transform(series)
# Matristeki sütun isimlerini, yani terimleri alır.
tfidf_tokens = tfidf_vectorized.get_feature_names_out()
# Matrisi bir veri çerçevesine dönüştürür ve sütun isimlerini terimler olarak atar.
df_tfidf_vect = pd.DataFrame(data=tfidf_wm.toarray(), columns=tfidf_tokens)

# Veri çerçevesini döndürür.
return df_tfidf_vect

In [5]:
from datetime import timedelta
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC, SVR
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from xgboost import XGBRegressor

def get_models(x, y, test_size=0.25, random_state=15, classification=False, average='binary', order_type='acc', info=False):
    """
        get_models adında bir fonksiyon tanımlanıyor. Bu fonksiyon, verilen x ve y değerleri için çeşitli makine öğrenmesi modellerini eğitir ve değerlendirir. Ayrıca test_size, random_state, classification, average, order_type ve info gibi parametreler alır. Bu parametrelerin anlamları şöyledir:
        test_size: Verinin ne kadarının test kümesi olarak ayrılacağını belirtir. 0 ile 1 arasında bir değer alır. Varsayılan değer 0.25'tir.
        random_state: Veriyi bölmeden önce karıştırmak için kullanılan rastgele sayı üretecinin tohumudur. Bir tam sayı veya None değeri alır. Varsayılan değer 15'tir.
        classification: Eğitilecek modellerin sınıflandırma mı yoksa regresyon mu olduğunu belirtir. Bir mantıksal değer alır. Varsayılan değer False'tur.
        average: Sınıflandırma modelleri için kullanılan metriklerin birden fazla sınıf için nasıl hesaplanacağını belirtir. 'binary', 'micro', 'macro', 'weighted' veya None değerlerinden birini alır. Varsayılan değer 'binary'dir.
        order_type: Eğitilen modellerin hangi metriğe göre sıralanacağını belirtir. 'acc', 'precision', 'recall', 'f1' veya 'time' değerlerinden birini alır. Varsayılan değer 'acc'dir.
        info: Eğitim sürecinde her model için bilgi mesajı yazdırılıp yazdırılmayacağını belirtir. Bir mantıksal değer alır. Varsayılan değer False'tur.

    """
    # Veriyi test_size oranında eğitim ve test kümelerine ayırır. random_state parametresi ile karıştırma işlemini kontrol eder.
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=random_state)
    # Eğitilen modellerin sonuçlarını saklamak için boş bir liste oluşturur.
    all_models = []
    # order_type parametresinin karşılık geldiği metriklerin indekslerini bir sözlükte tutar.
    order_types = {'acc': 1, 'precision': 2, 'recall': 3, 'f1': 4, 'time': 5}
    
    # classification parametresine göre eğitilecek modellerin listesini belirler. Bu liste, modelin adı ve nesnesi olarak ikililer içerir. Model nesneleri sklearn kütüphanesinden alınır. Bu örnekte, sınıflandırma için lojistik regresyon, k-en yakın komşu, karar ağacı, rastgele orman ve destek vektör makinesi modelleri; regresyon için ise doğrusal regresyon, ridge regresyon, lasso regresyon, elastik net regresyon ve destek vektör regresyonu modelleri kullanılmıştır. Ancak bu modeller değiştirilebilir.
    if classification:
        models = [
            ('LR', LogisticRegression(random_state=random_state)),
            ('KNN', KNeighborsClassifier()),
            ('CART', DecisionTreeClassifier(random_state=random_state)),
            ('RF', RandomForestClassifier(random_state=random_state)),
            ('SVM', SVC(gamma='auto', random_state=random_state)),
            ('GradientBoosting', GradientBoostingClassifier(random_state=random_state)),
            ("LightGBM", LGBMClassifier(random_state=random_state)),
            ("CatBoost", CatBoostClassifier(verbose=False, random_state=random_state))
        ]
    else:
        models = [
            ('LR', LinearRegression()),
            ("Ridge", Ridge()),
            ("Lasso", Lasso()),
            ("ElasticNet", ElasticNet()),
            ('KNN', KNeighborsRegressor()),
            ('CART', DecisionTreeRegressor()),
            ('RF', RandomForestRegressor()),
            ('SVR', SVR()),
            ('GBM', GradientBoostingRegressor()),
            ("XGBoost", XGBRegressor()),
            ("LightGBM", LGBMRegressor()),
            ("CatBoost", CatBoostRegressor(verbose=False))
        ]
    
    # Her bir model için bir döngü başlatır. name, modelin adı, model ise modelin nesnesidir.
    for name, model in models:
        # Eğitim süresini ölçmek için bir zamanlayıcı başlatır.
        start = timer()
        # info parametresi True ise, modelin eğitilmeye başlandığını ekrana yazdırır.
        if info:
            print(f"{name} is training")
        # Modeli eğitim verisi ile eğitir.
        model.fit(x_train, y_train)
        
        # classification parametresine göre değerlendirme metriklerini belirler. Sınıflandırma için doğruluk, kesinlik, geri çağırma ve f1 skoru; regresyon için ise eğitim ve test verileri için kök ortalama kare hata (RMSE) değerleri hesaplanır. Bu metrikler sklearn.metrics kütüphanesinden alınır. Ayrıca modelin adı ve eğitim süresi de kaydedilir.
        if classification:
            y_pred = model.predict(x_test)
            # Doğruluk skorunu hesaplar.
            acc_test = accuracy_score(y_test, y_pred)
            # Kesinlik skorunu hesaplar. average parametresi ile birden fazla sınıf için hesaplama yöntemi belirlenir.
            precision = precision_score(y_test, y_pred, average=average)
            # Geri çağırma skorunu hesaplar. average parametresi ile birden fazla sınıf için hesaplama yöntemi belirlenir.
            recall = recall_score(y_test, y_pred, average=average)
            # F1 skorunu hesaplar. average parametresi ile birden fazla sınıf için hesaplama yöntemi belirlenir.
            f1 = f1_score(y_test, y_pred, average=average)
            # Modelin adı, doğruluk, kesinlik, geri çağırma, f1 skoru ve eğitim süresi gibi değerleri bir sözlükte tutar.
            values = dict(name=name, acc_test=acc_test, precision=precision, recall=recall, f1=f1,
                          train_time=str(timedelta(seconds=(timer() - start)))[-15:])
        else:
            # Modeli hem eğitim hem de test verileri ile tahmin eder.
            y_pred_test = model.predict(x_test)
            y_pred_train = model.predict(x_train)
            # Eğitim ve test verileri için RMSE değerlerini hesaplar.
            rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
            rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
            # Modelin adı, eğitim ve test RMSE değerleri gibi değerleri bir sözlükte tutar.
            values = dict(name=name, RMSE_TRAIN=rmse_train, RMSE_TEST=rmse_test)
        
        # info parametresi True ise, modelin eğitim süresini ekrana yazdırır.
        if info:
            print(f"{name} is done in {timedelta(seconds=(timer() - start))}")
        # Sözlüğü all_models listesine ekler.
        all_models.append(values)

    # all_models listesini bir veri çerçevesine dönüştürür ve order_type parametresine göre sıralar. order_type parametresi 'time' ise küçükten büyüğe, diğer durumlarda büyükten küçüğe sıralar.
    all_models_df = pd.DataFrame(all_models).sort_values(by=list(all_models[0].keys())[order_types[order_type]],
                                                          ascending=order_type != 'time')
    # Tüm modellerin eğitildiğini ve order_type parametresine göre sıralandığını ekrana yazdırır.
    print(f"\nAll models are done --- ordered by {order_type}")
    # Veri çerçevesini markdown formatında ekrana yazdırır.
    print(all_models_df.to_markdown())
    # Fonksiyon None değerini döndürür.
    return None

In [6]:
#türkçe karakterleri ingilizce yapan fonksiyon
def tr_en_char_translate(series):
    series = series.str.replace('ı', 'i')
    series = series.str.replace('ü', 'u')
    series = series.str.replace('ö', 'o')
    series = series.str.replace('ğ', 'g')
    series = series.str.replace('ş', 's')
    series = series.str.replace('ç', 'c')
    return series

In [7]:
# learning-set.csv adlı bir dosyayı okur ve df adlı bir veri çerçevesine atar. Dosyanın ayırıcı karakteri noktalı virgüldür.
df = pd.read_csv("C:/Users/egeca/OneDrive/Masaüstü/İş Analtiği/FİNAL/learning-set.csv", sep=';')

# Veri çerçevesindeki topic adlı sütunun değerlerinin dağılımını ekrana yazdırır. topic, verilerin hangi konuya ait olduğunu gösteren bir sütundur.
print(df.topic.value_counts())

# Veri çerçevesine para adlı yeni bir sütun ekler. Bu sütun, topic sütununun değeri 'para' ise 1, değilse 0 olarak doldurulur. Böylece, verilerin para konulu olup olmadığını gösteren bir sütun oluşturulur.
df['para'] = [1 if x == 'para' else 0 for x in df.topic]

# Veri çerçevesine yaş adlı yeni bir sütun ekler. Bu sütun, topic sütununun değeri 'yaş' ise 1, değilse 0 olarak doldurulur. Böylece, verilerin yaş konulu olup olmadığını gösteren bir sütun oluşturulur.
df['yaş'] = [1 if x == 'yaş' else 0 for x in df.topic]

# Veri çerçevesine teklif adlı yeni bir sütun ekler. Bu sütun, topic sütununun değeri 'teklif' ise 1, değilse 0 olarak doldurulur. Böylece, verilerin teklif konulu olup olmadığını gösteren bir sütun oluşturulur.
df['teklif'] = [1 if x == 'teklif' else 0 for x in df.topic]

# Veri çerçevesinin ilk beş satırını ekrana yazdırır. Böylece, veri çerçevesinin yapısını ve içeriğini görebiliriz.
df.head()

yaş       150
para      150
teklif    150
Name: topic, dtype: int64


,topic,tweet,para,yaş,teklif
0,para,"Huzur, evlilik, para gordum galiba sugar d4dd...",1,0,0
1,para,Evlilik için ilk yatırımımı (çeyrek altın)yapm...,1,0,0
2,para,"@Winb1r yüzük, zenginlik, evlilik???? zengin k...",1,0,0
3,para,"Evlilik pahalılaştıkça , zina ucuzlayacaktır .",1,0,0
4,para,@lilililimarlen Çanım baa ölçüleri ve kriterle...,1,0,0


In [8]:
# Veri önişleme
df['islenmis_veri'] = preprocessing(df.tweet, remove_links=True, remove_mentions=True,
                                           remove_hashtag=True, lowercase=True, remove_numbers=True,
                                           remove_punctuation=True)

df['duygu_analizi'] = preprocessing(df.tweet, remove_links=True, remove_mentions=True,
                                           remove_hashtag=True, lowercase=True, remove_numbers=True,
                                           remove_punctuation=True)

df[['tweet', 'islenmis_veri', 'duygu_analizi']].head()

Lowercasing...
Lowercasing is done in 0:00:00.001054
Removing hashtags...
Removed in 0:00:00.001130
Removing mentions...
Removed in 0:00:00.001142
Removing links...
Removed in 0:00:00.002162
Removing numbers...
Removed in 0:00:00.002020
Removing punctuation...
Removed in 0:00:00.002806
Lowercasing...
Lowercasing is done in 0:00:00.000942
Removing hashtags...
Removed in 0:00:00.001353
Removing mentions...
Removed in 0:00:00.001786
Removing links...
Removed in 0:00:00.001567
Removing numbers...
Removed in 0:00:00.001535
Removing punctuation...
Removed in 0:00:00.001849


,tweet,islenmis_veri,duygu_analizi
0,"Huzur, evlilik, para gordum galiba sugar d4dd...",huzur evlilik para gordum galiba sugar dddymi...,huzur evlilik para gordum galiba sugar dddymi...
1,Evlilik için ilk yatırımımı (çeyrek altın)yapm...,evlilik için ilk yatırımımı çeyrek altınyapmış...,evlilik için ilk yatırımımı çeyrek altınyapmış...
2,"@Winb1r yüzük, zenginlik, evlilik???? zengin k...",yüzük zenginlik evlilik zengin koca buldum ga...,yüzük zenginlik evlilik zengin koca buldum ga...
3,"Evlilik pahalılaştıkça , zina ucuzlayacaktır .",evlilik pahalılaştıkça zina ucuzlayacaktır,evlilik pahalılaştıkça zina ucuzlayacaktır
4,@lilililimarlen Çanım baa ölçüleri ve kriterle...,çanım baa ölçüleri ve kriterleri yaz ben çalı...,çanım baa ölçüleri ve kriterleri yaz ben çalı...


In [9]:
#türkçe karakter değişimi
df['islenmis_veri_non_turkish_char'] = tr_en_char_translate(df['islenmis_veri'])
#gösterimi
df[['islenmis_veri', 'islenmis_veri_non_turkish_char']]

,islenmis_veri,islenmis_veri_non_turkish_char
0,huzur evlilik para gordum galiba sugar dddymi...,huzur evlilik para gordum galiba sugar dddymi...
1,evlilik için ilk yatırımımı çeyrek altınyapmış...,evlilik icin ilk yatirimimi ceyrek altinyapmis...
2,yüzük zenginlik evlilik zengin koca buldum ga...,yuzuk zenginlik evlilik zengin koca buldum ga...
3,evlilik pahalılaştıkça zina ucuzlayacaktır,evlilik pahalilastikca zina ucuzlayacaktir
4,çanım baa ölçüleri ve kriterleri yaz ben çalı...,canim baa olculeri ve kriterleri yaz ben cali...
...,...,...
445,evliliklan daha erken hayır,evliliklan daha erken hayir
446,pedofil yok ama peygamberimiz evlilik yaşı i...,pedofil yok ama peygamberimiz evlilik yasi i...
447,türkiye afganistanı örnek alsınkızlar okula g...,turkiye afganistani ornek alsinkizlar okula g...
448,yaşında taş gibi karı ne var bunda yaşındak...,yasinda tas gibi kari ne var bunda yasindak...


In [10]:
#nadir kelimelerin silinimi ve bunu deneme sütunu oluşturup ona eklemek
df['deneme'] = preprocessing(df.islenmis_veri, remove_rare_words=True, rare_limit=0)

Removing rare words...
There are 4117 words in the series
%0.0 of words appear lesser than rare limit (0) times
Removing rare words...
0 rare words removed
Removed in 0:00:00.012474


In [11]:
tf_idf_set = create_tfidf(df['islenmis_veri'])

In [12]:
#sırasıyla teste edip en başarılı modeli bulma
get_models(tf_idf_set, df.para, classification=True, info=True)
# para, en iyi KNN ile çalıştı.

get_models(tf_idf_set, df.teklif, classification=True, info=True)
# teklif, en iyi GradientBoosting ile çalıştı.

get_models(tf_idf_set, df.yaş, classification=True, info=True)
# yaş, en iyi CART ile çalıştı.

LR is training
LR is done in 0:00:00.344677
KNN is training
KNN is done in 0:00:00.259508
CART is training
CART is done in 0:00:00.113156
RF is training
RF is done in 0:00:00.518367
SVM is training


C:\Users\egeca\Python_Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM is done in 0:00:00.350977
GradientBoosting is training
GradientBoosting is done in 0:00:02.602159
LightGBM is training
[LightGBM] [Info] Number of positive: 108, number of negative: 229
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000239 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 369
[LightGBM] [Info] Number of data points in the train set: 337, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.320475 -> initscore=-0.751591
[LightGBM] [Info] Start training from score -0.751591
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

CatBoost is done in 0:00:11.361853

All models are done --- ordered by acc
|    | name             |   acc_test |   precision |    recall |       f1 | train_time     |
|---:|:-----------------|-----------:|------------:|----------:|---------:|:---------------|
|  4 | SVM              |   0.628319 |    0        | 0         | 0        | 0:00:00.350969 |
|  0 | LR               |   0.663717 |    1        | 0.0952381 | 0.173913 | 0:00:00.344668 |
|  6 | LightGBM         |   0.663717 |    0.555556 | 0.47619   | 0.512821 | 0:00:00.140466 |
|  3 | RF               |   0.725664 |    1        | 0.261905  | 0.415094 | 0:00:00.518358 |
|  7 | CatBoost         |   0.725664 |    0.789474 | 0.357143  | 0.491803 | 0:00:11.361845 |
|  2 | CART             |   0.761062 |    0.727273 | 0.571429  | 0.64     | 0:00:00.113148 |
|  5 | GradientBoosting |   0.778761 |    0.904762 | 0.452381  | 0.603175 | 0:00:02.602150 |
|  1 | KNN              |   0.80531  |    0.857143 | 0.571429  | 0.685714 | 0:00:00.2595

C:\Users\egeca\Python_Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM is done in 0:00:00.345699
GradientBoosting is training
GradientBoosting is done in 0:00:02.519223
LightGBM is training
[LightGBM] [Info] Number of positive: 118, number of negative: 219
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000161 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 369
[LightGBM] [Info] Number of data points in the train set: 337, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.350148 -> initscore=-0.618387
[LightGBM] [Info] Start training from score -0.618387
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

CatBoost is done in 0:00:11.033859

All models are done --- ordered by acc
|    | name             |   acc_test |   precision |   recall |       f1 | train_time     |
|---:|:-----------------|-----------:|------------:|---------:|---------:|:---------------|
|  4 | SVM              |   0.716814 |    0        |  0       | 0        | 0:00:00.345691 |
|  0 | LR               |   0.761062 |    1        |  0.15625 | 0.27027  | 0:00:00.112231 |
|  1 | KNN              |   0.858407 |    0.7      |  0.875   | 0.777778 | 0:00:00.062099 |
|  2 | CART             |   0.858407 |    0.785714 |  0.6875  | 0.733333 | 0:00:00.137146 |
|  3 | RF               |   0.884956 |    0.952381 |  0.625   | 0.754717 | 0:00:00.503698 |
|  6 | LightGBM         |   0.884956 |    0.88     |  0.6875  | 0.77193  | 0:00:00.096712 |
|  7 | CatBoost         |   0.893805 |    0.954545 |  0.65625 | 0.777778 | 0:00:11.033849 |
|  5 | GradientBoosting |   0.902655 |    0.862069 |  0.78125 | 0.819672 | 0:00:02.519213 |
LR is

C:\Users\egeca\Python_Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM is done in 0:00:00.306335
GradientBoosting is training
GradientBoosting is done in 0:00:02.468029
LightGBM is training
[LightGBM] [Info] Number of positive: 111, number of negative: 226
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000178 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 369
[LightGBM] [Info] Number of data points in the train set: 337, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.329377 -> initscore=-0.711005
[LightGBM] [Info] Start training from score -0.711005
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

CatBoost is done in 0:00:11.046082

All models are done --- ordered by acc
|    | name             |   acc_test |   precision |   recall |       f1 | train_time     |
|---:|:-----------------|-----------:|------------:|---------:|---------:|:---------------|
|  4 | SVM              |   0.654867 |    0        | 0        | 0        | 0:00:00.306327 |
|  0 | LR               |   0.707965 |    1        | 0.153846 | 0.266667 | 0:00:00.103829 |
|  6 | LightGBM         |   0.761062 |    0.730769 | 0.487179 | 0.584615 | 0:00:00.091531 |
|  3 | RF               |   0.787611 |    1        | 0.384615 | 0.555556 | 0:00:00.477651 |
|  1 | KNN              |   0.814159 |    0.875    | 0.538462 | 0.666667 | 0:00:00.061700 |
|  2 | CART             |   0.814159 |    0.8      | 0.615385 | 0.695652 | 0:00:00.135939 |
|  5 | GradientBoosting |   0.814159 |    0.909091 | 0.512821 | 0.655738 | 0:00:02.468021 |
|  7 | CatBoost         |   0.831858 |    0.954545 | 0.538462 | 0.688525 | 0:00:11.046074 |


In [13]:
#30.000 satırlık evlilik_1 veri setinin okunulması ve veri ön işlemesi
evlilik = pd.read_csv("C:/Users/egeca/OneDrive/Masaüstü/İş Analtiği/FİNAL/evlilik_1.csv", sep=',')

evlilik['temizlenmiş'] = preprocessing(evlilik.tweet, remove_links=True, remove_mentions=True,
                                           remove_hashtag=True, lowercase=True, remove_numbers=True,
                                           remove_punctuation=True)

evlilik['duygu_analizi'] = preprocessing(evlilik.tweet, remove_links=True, remove_mentions=True,
                                           remove_hashtag=True, lowercase=True, remove_numbers=True,
                                           remove_punctuation=True)

Lowercasing...
Lowercasing is done in 0:00:00.047459
Removing hashtags...
Removed in 0:00:00.026917
Removing mentions...
Removed in 0:00:00.038884
Removing links...
Removed in 0:00:00.072261
Removing numbers...
Removed in 0:00:00.093250
Removing punctuation...
Removed in 0:00:00.121729
Lowercasing...
Lowercasing is done in 0:00:00.041842
Removing hashtags...
Removed in 0:00:00.028282
Removing mentions...
Removed in 0:00:00.037766
Removing links...
Removed in 0:00:00.069508
Removing numbers...
Removed in 0:00:00.092101
Removing punctuation...
Removed in 0:00:00.122769


In [14]:
# Makine Öğrenmesi
tfidf_vectorized = TfidfVectorizer(analyzer='word')
tf_idf_donusturucu = tfidf_vectorized.fit(df['islenmis_veri'])
ogrenme_seti = tf_idf_donusturucu.transform(df['islenmis_veri'])
ogrenme_seti_vectoru = pd.DataFrame(data=ogrenme_seti.toarray(), columns=tf_idf_donusturucu.get_feature_names_out())
tahmin_seti = tf_idf_donusturucu.transform(evlilik.temizlenmiş)
tahmin_seti_vectoru = pd.DataFrame(data=tahmin_seti.toarray(), columns=tf_idf_donusturucu.get_feature_names_out())
print("bitti")

bitti


In [15]:
# Para sütununda KNN kullandık çünkü en başarılı oydu
knn_model_para = KNeighborsClassifier()
knn_model_para.fit(ogrenme_seti_vectoru, df.para)
evlilik['para'] = knn_model_para.predict(tahmin_seti_vectoru)

# Teklif  sütununda - GradientBoosting kullandık çünkü en başarılı oydu
gb_model_teklif = GradientBoostingClassifier()
gb_model_teklif.fit(ogrenme_seti_vectoru, df.teklif)
evlilik['teklif'] = gb_model_teklif.predict(tahmin_seti_vectoru)

# Yaş  sütununda - Decision Tree (CART) kullandık çünkü en başarılı oydu
cart_model_yas = DecisionTreeClassifier()
cart_model_yas.fit(ogrenme_seti_vectoru, df.yaş)
evlilik['yaş'] = cart_model_yas.predict(tahmin_seti_vectoru)

evlilik.head()

,tweet,Unnamed: 1,temizlenmiş,duygu_analizi,para,teklif,yaş
0,Üniversitemizin güllerinden Ekrem kardeşimizi ...,NaN,üniversitemizin güllerinden ekrem kardeşimizi ...,üniversitemizin güllerinden ekrem kardeşimizi ...,0,0,1
1,"Huzur, evlilik, para gordum galiba sugar d4dd...",NaN,huzur evlilik para gordum galiba sugar dddymi...,huzur evlilik para gordum galiba sugar dddymi...,1,0,0
2,tribünde önümüzde evlilik teklifi edildi. dald...,NaN,tribünde önümüzde evlilik teklifi edildi daldı...,tribünde önümüzde evlilik teklifi edildi daldı...,0,1,0
3,@Komucuuu Nasıl önleyebiliriz ? sanırım evlili...,NaN,nasıl önleyebiliriz sanırım evlilik öncesi b...,nasıl önleyebiliriz sanırım evlilik öncesi b...,1,0,0
4,@Winb1r Yüzük nişan evlilik bu kadar denk gelm...,NaN,yüzük nişan evlilik bu kadar denk gelmez evre...,yüzük nişan evlilik bu kadar denk gelmez evre...,0,1,0


In [16]:
# afet, cevre ve iklimin 0 olduğu satırlar
len(evlilik[(evlilik.yaş == 0) & (evlilik.teklif == 0) & (evlilik.para == 0)])

14796

In [18]:
#duygu analizi
evlilik['duygu_analizi'] = evlilik.duygu_analizi.astype(str)
evlilik[['label', 'score']] = sentiment(evlilik.duygu_analizi)

Sentiment analysis of 100 tweets is done
Sentiment analysis of 200 tweets is done
Sentiment analysis of 300 tweets is done
Sentiment analysis of 400 tweets is done
Sentiment analysis of 500 tweets is done
Sentiment analysis of 600 tweets is done
Sentiment analysis of 700 tweets is done
Sentiment analysis of 800 tweets is done
Sentiment analysis of 900 tweets is done
Sentiment analysis of 1000 tweets is done
Sentiment analysis of 1100 tweets is done
Sentiment analysis of 1200 tweets is done
Sentiment analysis of 1300 tweets is done
Sentiment analysis of 1400 tweets is done
Sentiment analysis of 1500 tweets is done
Sentiment analysis of 1600 tweets is done
Sentiment analysis of 1700 tweets is done
Sentiment analysis of 1800 tweets is done
Sentiment analysis of 1900 tweets is done
Sentiment analysis of 2000 tweets is done
Sentiment analysis of 2100 tweets is done
Sentiment analysis of 2200 tweets is done
Sentiment analysis of 2300 tweets is done
Sentiment analysis of 2400 tweets is done
S

Sentiment analysis of 19500 tweets is done
Sentiment analysis of 19600 tweets is done
Sentiment analysis of 19700 tweets is done
Sentiment analysis of 19800 tweets is done
Sentiment analysis of 19900 tweets is done
Sentiment analysis of 20000 tweets is done
Sentiment analysis of 20100 tweets is done
Sentiment analysis of 20200 tweets is done
Sentiment analysis of 20300 tweets is done
Sentiment analysis of 20400 tweets is done
Sentiment analysis of 20500 tweets is done
Sentiment analysis of 20600 tweets is done
Sentiment analysis of 20700 tweets is done
Sentiment analysis of 20800 tweets is done
Sentiment analysis of 20900 tweets is done
Sentiment analysis of 21000 tweets is done
Sentiment analysis of 21100 tweets is done
Sentiment analysis of 21200 tweets is done
Sentiment analysis of 21300 tweets is done
Sentiment analysis of 21400 tweets is done
Sentiment analysis of 21500 tweets is done
Sentiment analysis of 21600 tweets is done
Sentiment analysis of 21700 tweets is done
Sentiment a

In [20]:
#ödev formatı
evlilik[['tweet','para','teklif','yaş','label', 'score']]

,tweet,para,teklif,yaş,label,score
0,Üniversitemizin güllerinden Ekrem kardeşimizi ...,0,0,1,negative,0.943523
1,"Huzur, evlilik, para gordum galiba sugar d4dd...",1,0,0,negative,0.753243
2,tribünde önümüzde evlilik teklifi edildi. dald...,0,1,0,negative,0.668888
3,@Komucuuu Nasıl önleyebiliriz ? sanırım evlili...,1,0,0,negative,0.666369
4,@Winb1r Yüzük nişan evlilik bu kadar denk gelm...,0,1,0,negative,0.543826
...,...,...,...,...,...,...
29995,"Evlilik aşkı öldürür dediler daha çok sevdik, ...",1,0,0,negative,0.965834
29996,@SinemM070327 Evlilik sadece aşkı değil ruhund...,0,0,0,negative,0.738472
29997,Evlilik aşkı değil herşeyi öldürür,0,0,0,positive,0.729743
29998,Evlilik aşkı öldürür diyorlardı ama ben inanmı...,0,0,0,positive,0.697920


In [21]:
#ödev veri setinin oluşturulması
ödev = evlilik[['tweet','para','teklif','yaş','label', 'score']]

In [22]:
ödev.head()

,tweet,para,teklif,yaş,label,score
0,Üniversitemizin güllerinden Ekrem kardeşimizi ...,0,0,1,negative,0.943523
1,"Huzur, evlilik, para gordum galiba sugar d4dd...",1,0,0,negative,0.753243
2,tribünde önümüzde evlilik teklifi edildi. dald...,0,1,0,negative,0.668888
3,@Komucuuu Nasıl önleyebiliriz ? sanırım evlili...,1,0,0,negative,0.666369
4,@Winb1r Yüzük nişan evlilik bu kadar denk gelm...,0,1,0,negative,0.543826


In [23]:
#ödevi kaydetmek
ödev.to_csv('ödev-teslim.csv', index=False)